In [ ]:
"""

from django.contrib.gis.db import models as geomodels
from cities_light.abstract_models import (
    AbstractCity,
    AbstractCountry,
    AbstractRegion,
    AbstractSubRegion,
)
from cities_light.receivers import connect_default_signals


class Country(AbstractCountry):
    boundary = geomodels.MultiPolygonField(null=True, blank=True)

connect_default_signals(Country)

class Region(AbstractRegion):
    boundary = geomodels.MultiPolygonField(null=True, blank=True)

connect_default_signals(Region)

class SubRegion(AbstractSubRegion):
    boundary = geomodels.MultiPolygonField(null=True, blank=True)

connect_default_signals(SubRegion)

class City(AbstractCity):
    boundary = geomodels.MultiPolygonField(null=True, blank=True)

connect_default_signals(City)
"""


In [ ]:
!pip install pygadm geonames-lib
!pip install matplotlib geopandas folium
!pip install shapely


In [ ]:
import urllib.request as request
import zipfile
import os

# URL of the geonames dataset
url = 'http://download.geonames.org/export/dump/allCountries.zip'

# Path where you want to save the zip file
zip_path = 'geonames/allCountries.zip'

# Download and save the file
with request.urlopen(url) as response, open(zip_path, 'wb') as out_file:
    data = response.read()
    out_file.write(data)

    
# Extract the file from the zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extract('geonames/allCountries.txt')
    
print("Downloaded and saved dataset.")


In [ ]:
import geonames

geonames_data = geonames.GeoNames('geonames/allCountries.txt').data


In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import geopandas as gpd

import setup_django
from geoprod.cities.models import City, Region, Country, SubRegion
import pygadm

In [ ]:
def get_all_boundaries(content_level):
    """
    Fetches and concatenates administrative boundaries for all countries.

    Args:
    content_level (int): The administrative level to fetch for each country.

    Returns:
    GeoDataFrame: A GeoDataFrame containing boundaries for all countries.
    """
    # Initialize an empty GeoDataFrame
    all_boundaries_gdf = gpd.GeoDataFrame()

    # Iterate over countries
    countries = Country.objects.all()
    for country in tqdm(countries, desc='Fetching Boundaries'):
        try:
            # Fetch administrative boundaries
            boundaries = pygadm.AdmItems(admin=country.code3, content_level=content_level)
            boundaries_gdf = gpd.GeoDataFrame.from_features(boundaries).set_crs(epsg=4326, inplace=True)

            # Add a column for the country code
            boundaries_gdf['country_code'] = country.code3

            # Concatenate with the main GeoDataFrame
            all_boundaries_gdf = pd.concat([all_boundaries_gdf, boundaries_gdf], ignore_index=True)
        except Exception as e:
            print(f"Error processing country {country.name}: {e}")

    return all_boundaries_gdf



In [ ]:
from shapely.geometry import Point
import geopandas as gpd

def get_location_row(geonames_data, geoname_id):
    """
    Fetches the row from the GeoNames dataset for a given geoname_id and creates a Point object.

    Args:
    geonames_data (DataFrame): The GeoNames DataFrame.
    geoname_id (int): The geoname_id of the country or entity.

    Returns:
    DataFrame row, Point: The corresponding row from the DataFrame and a Point object.
    """
    # Fetch the row corresponding to the geoname_id
    entity_row = geonames_data[geonames_data['geonameid'] == geoname_id]

    if not entity_row.empty:
        # Extract latitude and longitude
        latitude = entity_row.iloc[0].latitude
        longitude = entity_row.iloc[0].longitude

        # Create a Point object
        point = Point(longitude, latitude)

        return entity_row, point
    else:
        return None, None



In [ ]:
# Filtering for Countries boundaries
all_countries_boundaries_gdf = get_all_boundaries(0)

In [ ]:
from tqdm.notebook import tqdm
from shapely.geometry import MultiPolygon
from django.contrib.gis.geos import GEOSGeometry


# Assuming get_location_row and all_countries_boundaries_gdf are defined

countries = Country.objects.all()
for country in tqdm(countries):
    country_row, point = get_location_row(geonames_data, country.geoname_id)

    if point:
        country_boundary = all_countries_boundaries_gdf[all_countries_boundaries_gdf.contains(point)]
        if not country_boundary.empty:
            # Ensure the geometry is a MultiPolygon
            geometry = country_boundary.geometry.iloc[0]
            if isinstance(geometry, MultiPolygon):
                country.boundary = GEOSGeometry(geometry.wkt)
                country.save()
            else:
                print(f"Geometry for {country} is not a MultiPolygon.")
        else:
            print(f"No boundary found for {country}.")
    else:
        print(f"No location data for {country}.")


In [ ]:
# Regions boundaries
all_regions_boundaries_gdf = get_all_boundaries(1)

In [ ]:
from tqdm.notebook import tqdm
from shapely.geometry import MultiPolygon
from django.contrib.gis.geos import GEOSGeometry

regions = Region.objects.all()  # Adjust the filter as needed
for region in tqdm(regions):
    region_row, point = get_location_row(geonames_data, region.geoname_id)

    if point:
        region_boundary = all_regions_boundaries_gdf[all_regions_boundaries_gdf.contains(point)]
        if not region_boundary.empty:
            # Ensure the geometry is a MultiPolygon
            geometry = region_boundary.geometry.iloc[0]
            if isinstance(geometry, MultiPolygon):
                region.boundary = GEOSGeometry(geometry.wkt)
                region.save()
            else:
                print(f"Geometry for {region} is not a MultiPolygon.")
        else:
            print(f"No boundary found for {region}.")
    else:
        print(f"No location data for {region}.")


In [ ]:
# SubRegions boundaries
all_subregions_boundaries_gdf = get_all_boundaries(2)


In [ ]:
from tqdm.notebook import tqdm
from shapely.geometry import MultiPolygon
from django.contrib.gis.geos import GEOSGeometry

subregions = SubRegion.objects.all()  # Adjust the filter as needed
for subregion in tqdm(subregions):
    subregion_row, point = get_location_row(geonames_data, subregion.geoname_id)

    if point:
        subregion_boundary = all_subregions_boundaries_gdf[all_subregions_boundaries_gdf.contains(point)]
        if not subregion_boundary.empty:
            # Ensure the geometry is a MultiPolygon
            geometry = subregion_boundary.geometry.iloc[0]
            if isinstance(geometry, MultiPolygon):
                subregion.boundary = GEOSGeometry(geometry.wkt)
                subregion.save()
            else:
                print(f"Geometry for {subregion} is not a MultiPolygon.")
        else:
            print(f"No boundary found for {subregion}.")
    else:
        print(f"No location data for {subregion}.")


In [ ]:
from tqdm.notebook import tqdm
from shapely.geometry import MultiPolygon
from django.contrib.gis.geos import GEOSGeometry

cities = City.objects.all()  # Adjust the filter as needed
for city in tqdm(cities):
    city_row, point = get_location_row(geonames_data, city.geoname_id)

    if point:
        city_boundary = all_cities_boundaries_gdf[all_cities_boundaries_gdf.contains(point)]
        if not city_boundary.empty:
            # Ensure the geometry is a MultiPolygon
            geometry = city_boundary.geometry.iloc[0]
            if isinstance(geometry, MultiPolygon):
                city.boundary = GEOSGeometry(geometry.wkt)
                city.save()
            else:
                print(f"Geometry for {city} is not a MultiPolygon.")
        else:
            print(f"No boundary found for {city}.")
    else:
        print(f"No location data for {city}.")
